In [299]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from pprint import pprint
from collections import defaultdict
import datetime as dt
from dateutil.parser import parse
import pickle
import re
import time
import os

# Individual game page scraper

## Helper functions

In [419]:
def grab_soup(url):
    """Takes a url and returns a BeautifulSoup object"""
    
    response = requests.get(url)
    
    assert (response.status_code == 200), "Problem with url request! %s throws %s" % (url, response.status_code)   #checking that it worked
    
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    return soup


def get_score(soup):
    """Takes a BeautifulSoup object of an individual game page and returns the aggregate
    critic score of the game. If there is no critic score, returns None."""
    
    scorebox = soup.find(id='coreGameRank')('div')
    score = scorebox[5].text    #position of numerical score ("..." if no score listed)
    
    if score == "...":       #avoid casting a missing score to an int
        return None
    else:
        return int(score)
    

def get_title_console(soup):
    """Takes a BeautifulSoup object of an individual game page and returns the
    title and console of the game as a tuple (title, platform)"""
    header = soup.find(class_="niceHeaderTitle")("a")
    gametitle = header[0].text
    console = header[1].text
    
    return gametitle, console
    
    
def parse_game_data(soup):
    """Takes a BeautifulSoup object of a game's info page and returns a dictionary
    with the characteristics and target value (critic score) for the game"""
    
    gameinfo = defaultdict()     #Dictionary to hold the info for this game
    
    #Grabs the info from the coreGameRelease section of page
    releaseinfo = soup.find(id='coreGameRelease')('div')
    
    num_cells = len(releaseinfo)
    current_cell = 0
    
    while current_cell < (num_cells - 1):
        field = releaseinfo[current_cell].text.replace('\xa0', ' ').lower()
        value = releaseinfo[current_cell + 1].text.replace('\xa0', ' ')
    
        gameinfo[field] = value
        
        current_cell += 2
    
    #Grabs the info from the 'coreGameGenre' section of page
    genreinfo = soup.find(id='coreGameGenre')('div')

    num_cells = len(genreinfo)
    
    #The first cell in this section is either blank or garbage. Blank cells are followed by
    #a garbage cell. This if-statement skips the 1 garbage cell, or the blank cell AND the following
    #garbage cell.
    if genreinfo[0].text == "":
        current_cell = 2
    else:
        current_cell = 1

    while current_cell < (num_cells - 1):
        field = genreinfo[current_cell].text.replace('\xa0', ' ').lower()
        value = genreinfo[current_cell + 1].text.replace('\xa0', ' ')
        
        gameinfo[field] = value
    
        current_cell += 2
    
    #helper function - retrieves critic score (our target) if available, else 'None'
    score = get_score(soup)
    gameinfo['score'] = score
    
    #helper function - retrieves the title and console of the game
    title, console = get_title_console(soup)
    gameinfo['title'] = title
    gameinfo['console'] = console
    
    return gameinfo

## All together now - single page scraper function

In [366]:
def single_page_scrape(url):
    """Takes the url for a single game page on MobyGames, scrapes the html, parses the data,
    and returns a dict containing the characteristics and target value for that game"""
    soup = grab_soup(url)
    data = parse_game_data(soup)
    
    return data

**Testing the function:**

In [363]:
test = single_page_scrape('http://www.mobygames.com/game/switch/fire-emblem-warriors')
pprint(test)

78
defaultdict(None,
            {'also for': 'New Nintendo 3DS | Combined View',
             'art': 'Anime / Manga',
             'console': 'Nintendo Switch',
             'developed by': 'Intelligent Systems Co., Ltd., Omega Force, Team '
                             'Ninja',
             'esrb rating': 'Teen',
             'gameplay': 'Hack and Slash',
             'genre': 'Action',
             'interface': 'Direct Control',
             'official site': 'Fire Emblem: Warriors',
             'perspective': 'Behind view',
             'published by': 'Nintendo of America Inc.',
             'released': 'Oct 20, 2017',
             'score': 78,
             'setting': 'Fantasy',
             'title': 'Fire Emblem: Warriors'})


# Scrape urls for every game on a console

## Helper functions

In [124]:
def get_game_quantity(url):
    """This function takes the url of a game list on moby games and returns an int of how many total
    games are in that category"""
    soup = grab_soup(url)
    
    header = soup.find(class_="mobHeaderItems").text  #returns string '(items 1-25 of N)' where N is total games
    
    total_number = int(header[15:-1]) #pulls out only the total number of games and casts as int

    return total_number


def get_urls(url):
    """This function looks at a list of games on MobyGames.com (url specified as input parameter)
    and returns a list of urls for every individual game on that list"""
    
    url_list = []      #series to hold all the game page urls I'm pulling

    soup = grab_soup(url)

    gametable = soup.find(id="mof_object_list")    #grabs the main table with all the games
    tablerows = gametable.find("tbody")("tr")      #finds all "tr" tagged sections in the "tbody" of the gametable

    for row in tablerows:       #the first link in each row is what we want
        raw_tag = row.find("a")
        url_tail = raw_tag['href']      #pulls the relative url
        full_url = "www.mobygames.com" + url_tail

        url_list.append(full_url)

    return url_list

## All together now - function to collect game urls for any console

In [125]:
def get_game_urls_console(base_url='http://www.mobygames.com/browse/games/switch'):
    """Takes the url for a console's game list on MobyGames (defaults to Nintendo Switch)
    and returns a list of urls for every game on the console"""
    
    game_urls = []     #this will store the url list for return

    url_head = base_url + '/offset,'
    offset = 0       #this is plugged into the url to progress through pages of the games list
    url_tail = '/so,0a/list-games/'

    total_games = get_game_quantity(url_head + str(offset) + url_tail) #figure out how many games are in category

    while offset <= (total_games):
        url = url_head + str(offset) + url_tail

        games_on_page = get_urls(url)
        game_urls += games_on_page

        offset += 25
    
    return game_urls

## Generate URL lists

**Generating lists for Nintendo Switch, PS4, XBox One**

In [130]:
ns_list = get_game_urls_console()     #nintendo switch is the default for that function

In [ ]:
ps_list = get_game_urls_console(base_url='http://www.mobygames.com/browse/games/playstation-4')

In [131]:
xb_list = get_game_urls_console(base_url='http://www.mobygames.com/browse/games/xbox-one')

**Saving each list as pickle to avoid rerunning the scrapes above**

In [134]:
with open('ns_urls.pk1', 'wb') as f:
    pickle.dump(ns_list, f)

with open('ps_urls.pkl', 'wb') as f:
    pickle.dump(ps_list, f) 

with open('xb_urls.pkl', 'wb') as f:
    pickle.dump(xb_list, f)

# Scrape and aggregate data from each game url

## Create empty dataframe with desired columns

In [ ]:
fields = ['title','console','released','genre','gameplay','perspective',
              'setting','developed by','published by','esrb rating','score']

df = pd.DataFrame(columns=fields)

## Run scrape all game pages and populate dataframe

In [422]:
master_list = ns_list + ps_list + xb_list

for row, url in enumerate(master_list):
    gameinfo = single_page_scrape('http://' + url)
    
    for key,value in gameinfo.items():
        if key in list(df):
            df.loc[row,key] = value

In [427]:
df   #testing that it worked

,title,console,released,genre,gameplay,perspective,setting,developed by,published by,esrb rating,score
0,10 Second Run,Nintendo Switch,"Dec 21, 2017",Action,Platform,Side view,NaN,"G-mode Co., Ltd.","Blue Print Inc, KaeruPanda Inc.",NaN,None
1,1-2-Switch,Nintendo Switch,"Mar 03, 2017",Action,"Mini-Games, Party Game","1st-person, Audio game",NaN,Nintendo EPD,Nintendo of America Inc.,Everyone 10+,48
2,36 Fragments of Midnight,Nintendo Switch,"Sep 14, 2017",Action,Platform,Side view,NaN,Petite Games,Ratalaika Games S.L.,Everyone,None
3,60 Seconds!,Nintendo Switch,"Dec 18, 2017","Action, Adventure",NaN,Bird's-eye view,Post-Apocalyptic,Robot Gentleman,Robot Gentleman,NaN,None
4,88 Heroes,Nintendo Switch,"Oct 10, 2017",Action,Platform,Side view,NaN,Bitmap Bureau Ltd.,Rising Star Games Inc.,Everyone 10+,None
5,Aces of the Luftwaffe: Squadron,Nintendo Switch,"Nov 17, 2017",Action,"Arcade, Shooter",Top-down,NaN,Handy-Games GmbH,Handy-Games GmbH,Everyone 10+,None
6,Acorn Tactics,Nintendo Switch,"Dec 07, 2017",Strategy/Tactics,Managerial / Business Simulation,Bird's-eye view,Sci-Fi / Futuristic,TACS Games,TACS Games,NaN,None
7,Aero Fighters 2,Nintendo Switch,"Aug 03, 2017",Action,"Arcade, Shooter",Top-down,NaN,"Video System Co., Ltd.",HAMSTER Corporation,NaN,None
8,Alpha Mission II,Nintendo Switch,"Apr 06, 2017",Action,Shooter,Top-down,Sci-Fi / Futuristic,SNK Corporation,HAMSTER Corporation,Everyone,None
9,Ambition of the Slimes,Nintendo Switch,"Dec 28, 2017",Role-Playing (RPG),Tactical RPG,NaN,Fantasy,"Altair Works Corporation, Flyhigh Works Co., Ltd.",CIRCLE Entertainment Ltd.,NaN,None


## Save full raw dataframe as pickle for safekeeping

In [425]:
with open('allgames.pk1', 'wb') as f:     #whole dataset pickled!
    pickle.dump(df, f)

# Isolate rows with target value and export for analysis

In [469]:
mask = df['score'].notnull()
df_scores = df[mask]

with open('score_df.pk1', 'wb') as f:
    pickle.dump(df_scores, f)

# Appendix: Unusued Code

## Alternate method for getting game title

In [ ]:
def get_title(soup):
    """Takes a BeautifulSoup object of an individual game page and returns the
    cleaned up title of the game as a string"""
    pagetitle = soup.title.text
    clean_game_title = re.sub(r'\ for .*MobyGames', '', pagetitle)
    return clean_game_title

# Sandbox

In [351]:
response = requests.get('http://www.mobygames.com/game/switch/super-mario-odyssey')

assert (response.status_code == 200), "Problem with url request!"   #checking that it worked

page = response.text
soup = BeautifulSoup(page, "lxml")

scorebox = soup.find(id='coreGameRank')('div')

score = int(scorebox[5].text)

# if scorebox:
#     print(int(score_raw.text))
# else:
#     return None

In [296]:
header = soup.find(class_="niceHeaderTitle")("a")

gametitle = header[0].text
platform = header[1].text

print(gametitle)
print(platform)

Super Mario Odyssey
Nintendo Switch


In [181]:
genreinfo = soup.find(id='coreGameGenre')('div')

gameinfo = defaultdict()

num_cells = len(releaseinfo)
current_cell = 0

while current_cell < num_cells:
    field = releaseinfo[current_cell].text
    value = releaseinfo[current_cell + 1].text
    
    gameinfo[field] = value
    
    current_cell += 2

pprint(gameinfo)

defaultdict(None,
            {'Developed by': 'Nintendo\xa0EPD',
             'Official Site': 'Super Mario Odyssey',
             'Platform': 'Nintendo Switch',
             'Published by': 'Nintendo\xa0of\xa0America\xa0Inc.',
             'Released': 'Oct 27, 2017'})


In [158]:
score_raw = soup.find(class_='fr scoreBoxBig scoreHi')   #target value! aggregate critic score
gameinfo['score'] = int(score_raw.text)

pprint(gameinfo)

defaultdict(None,
            {'console': 'Nintendo Switch',
             'date': datetime.datetime(2017, 10, 27, 0, 0),
             'developer': 'Nintendo EPD',
             'esrb': 'Everyone 10+',
             'gameplay': 'Platform',
             'genre': 'Action',
             'publisher': 'Nintendo of America Inc.',
             'score': 96,
             'setting': 'Fantasy',
             'title': 'Super Mario Odyssey',
             'view': 'Behind view'})


In [376]:
url = 'http://www.mobygames.com/game/playstation-4/fat-city_'

response = requests.get(url)
    
assert (response.status_code == 200), "Problem with url request! %s" % (url,)   #checking that it worked
    
page = response.text
soup = BeautifulSoup(page, "lxml")
print(soup)

<!DOCTYPE html>
<html>
<head>
<title>Fat City for PlayStation 4 (2015) - MobyGames</title>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<base target="_top"/>
<meta content="" name="description"/>
<meta content="MobyGames" property="og:site_name"/>
<meta content="website" property="og:type"/>
<meta content="http://www.mobygames.com/images/covers/l/393259-fat-city-playstation-4-front-cover.png" property="og:image"/>
<meta content="Fat City for PlayStation 4 (2015) - MobyGames" property="og:title"/>
<meta content="" property="og:description"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="@MobyGames" name="twitter:site"/>
<meta content="Fat City for PlayStation 4 (2015) - MobyGames" name="twitter:title"/>
<meta content="" name="twitter:description"/>
<meta content="http://www.mobygames.com/images/covers/l/393259-fat-city-playstation-4-front-cover.pn

In [381]:
for key, value in enumerate(master_list):
    print("key: " + str(key) + "     " + "value: " + str(value))

key: 0     value: www.mobygames.com/game/switch/10-second-run
key: 1     value: www.mobygames.com/game/switch/1-2-switch
key: 2     value: www.mobygames.com/game/switch/36-fragments-of-midnight
key: 3     value: www.mobygames.com/game/switch/60-seconds
key: 4     value: www.mobygames.com/game/switch/88-heroes
key: 5     value: www.mobygames.com/game/switch/aces-of-the-luftwaffe-squadron
key: 6     value: www.mobygames.com/game/switch/acorn-tactics
key: 7     value: www.mobygames.com/game/switch/aero-fighters-2
key: 8     value: www.mobygames.com/game/switch/alpha-mission-ii
key: 9     value: www.mobygames.com/game/switch/ambition-of-the-slimes
key: 10     value: www.mobygames.com/game/switch/antiquia-lost
key: 11     value: www.mobygames.com/game/switch/arms
key: 12     value: www.mobygames.com/game/switch/art-of-fighting
key: 13     value: www.mobygames.com/game/switch/art-of-fighting-3-the-path-of-the-warrior
key: 14     value: www.mobygames.com/game/switch/astro-bears-party
key: 15 

key: 1443     value: www.mobygames.com/game/playstation-4/final-fantasy-xv
key: 1444     value: www.mobygames.com/game/playstation-4/final-fantasy-xv-assassins-festival
key: 1445     value: www.mobygames.com/game/playstation-4/final-fantasy-xv-deluxe-edition
key: 1446     value: www.mobygames.com/game/playstation-4/final-fantasy-xv-digital-premium-edition
key: 1447     value: www.mobygames.com/game/playstation-4/final-fantasy-xv-episode-gladiolus
key: 1448     value: www.mobygames.com/game/playstation-4/final-fantasy-xv-episode-ignis
key: 1449     value: www.mobygames.com/game/playstation-4/final-fantasy-xv-episode-prompto-
key: 1450     value: www.mobygames.com/game/playstation-4/final-fantasy-x-x-2-hd-remaster
key: 1451     value: www.mobygames.com/game/playstation-4/final-fantasy-x-x-2-hd-remaster-limited-edition
key: 1452     value: www.mobygames.com/game/playstation-4/final-horizon
key: 1453     value: www.mobygames.com/game/playstation-4/final-horizon-dark-galaxy
key: 1454     va

key: 2442     value: www.mobygames.com/game/playstation-4/observer_
key: 2443     value: www.mobygames.com/game/playstation-4/oceanhorn-monster-of-uncharted-seas
key: 2444     value: www.mobygames.com/game/playstation-4/octodad-dadliest-catch
key: 2445     value: www.mobygames.com/game/playstation-4/oddworld-abes-oddysee-new-n-tasty
key: 2446     value: www.mobygames.com/game/playstation-4/odin-sphere-leifthrasir
key: 2447     value: www.mobygames.com/game/playstation-4/oh-my-godheads
key: 2448     value: www.mobygames.com/game/playstation-4/ohsir-the-insult-simulator
key: 2449     value: www.mobygames.com/game/playstation-4/okage-shadow-king
key: 2450     value: www.mobygames.com/game/playstation-4/kami
key: 2451     value: www.mobygames.com/game/playstation-4/ok-ko-lets-play-heroes
key: 2452     value: www.mobygames.com/game/playstation-4/old-time-hockey
key: 2453     value: www.mobygames.com/game/playstation-4/olliolli
key: 2454     value: www.mobygames.com/game/playstation-4/olliol

key: 3442     value: www.mobygames.com/game/playstation-4/shoppe-keep
key: 3443     value: www.mobygames.com/game/playstation-4/shovel-knight
key: 3444     value: www.mobygames.com/game/playstation-4/shovel-knight-specter-of-torment
key: 3445     value: www.mobygames.com/game/playstation-4/shred-it
key: 3446     value: www.mobygames.com/game/playstation-4/shu
key: 3447     value: www.mobygames.com/game/playstation-4/shusse-zum
key: 3448     value: www.mobygames.com/game/playstation-4/shutshimi-seriously-swole
key: 3449     value: www.mobygames.com/game/playstation-4/siegecraft-commander
key: 3450     value: www.mobygames.com/game/playstation-4/silence
key: 3451     value: www.mobygames.com/game/playstation-4/silver-case
key: 3452     value: www.mobygames.com/game/playstation-4/sine-mora-ex
key: 3453     value: www.mobygames.com/game/playstation-4/singstar__
key: 3454     value: www.mobygames.com/game/playstation-4/singstar-die-eisknigin-vllig-unverforen
key: 3455     value: www.mobygam

key: 4441     value: www.mobygames.com/game/xbox-one/7-days-to-die-the-detention-squad
key: 4442     value: www.mobygames.com/game/xbox-one/7-days-to-die-the-walking-dead-skin-pack
key: 4443     value: www.mobygames.com/game/xbox-one/7-days-to-die-the-walking-dead-skin-pack-2
key: 4444     value: www.mobygames.com/game/xbox-one/7-days-to-die-the-walking-dead-skin-pack-bundle
key: 4445     value: www.mobygames.com/game/xbox-one/88-heroes
key: 4446     value: www.mobygames.com/game/xbox-one/8-bit-adventure-anthology-volume-i
key: 4447     value: www.mobygames.com/game/xbox-one/8days
key: 4448     value: www.mobygames.com/game/xbox-one/911-operator
key: 4449     value: www.mobygames.com/game/xbox-one/99vidas
key: 4450     value: www.mobygames.com/game/xbox-one/aaero
key: 4451     value: www.mobygames.com/game/xbox-one/aarus-awakening
key: 4452     value: www.mobygames.com/game/xbox-one/aban-hawkins-the-1001-spikes-the-temple-of-the-dead-mourns-the-l
key: 4453     value: www.mobygames.com/

key: 5441     value: www.mobygames.com/game/xbox-one/incredible-adventures-of-van-helsing-extended-edition
key: 5442     value: www.mobygames.com/game/xbox-one/incredible-adventures-of-van-helsing-iii
key: 5443     value: www.mobygames.com/game/xbox-one/incredible-adventures-of-van-helsing-ii
key: 5444     value: www.mobygames.com/game/xbox-one/incredible-adventures-of-van-helsing
key: 5445     value: www.mobygames.com/game/xbox-one/industry-giant-ii__
key: 5446     value: www.mobygames.com/game/xbox-one/infinite-mini-golf
key: 5447     value: www.mobygames.com/game/xbox-one/infinity-runner
key: 5448     value: www.mobygames.com/game/xbox-one/injustice-2
key: 5449     value: www.mobygames.com/game/xbox-one/injustice-2-black-manta
key: 5450     value: www.mobygames.com/game/xbox-one/injustice-2-brainiac
key: 5451     value: www.mobygames.com/game/xbox-one/injustice-2-darkseid
key: 5452     value: www.mobygames.com/game/xbox-one/injustice-2-deluxe-edition
key: 5453     value: www.mobygam

key: 6440     value: www.mobygames.com/game/xbox-one/rocksmith-all-new-2014-edition-green-day-jesus-of-suburbia
key: 6441     value: www.mobygames.com/game/xbox-one/rocksmith-all-new-2014-edition-green-day-know-your-enemy
key: 6442     value: www.mobygames.com/game/xbox-one/rocksmith-all-new-2014-edition-green-day-song-pack-iii
key: 6443     value: www.mobygames.com/game/xbox-one/rocksmith-all-new-2014-edition-green-day-welcome-to-paradise
key: 6444     value: www.mobygames.com/game/xbox-one/rocksmith-all-new-2014-edition-halestorm-bad-romance
key: 6445     value: www.mobygames.com/game/xbox-one/rocksmith-all-new-2014-edition-halestorm-i-get-off
key: 6446     value: www.mobygames.com/game/xbox-one/rocksmith-all-new-2014-edition-halestorm-i-miss-the-misery
key: 6447     value: www.mobygames.com/game/xbox-one/rocksmith-all-new-2014-edition-halestorm-love-bites-so-do-i
key: 6448     value: www.mobygames.com/game/xbox-one/rocksmith-all-new-2014-edition-halestorm-mz-hyde
key: 6449     value

key: 7440     value: www.mobygames.com/game/xbox-one/world-of-tanks-monster-kaiju-loaded-bundle
key: 7441     value: www.mobygames.com/game/xbox-one/world-of-tanks-monster-lycan-loaded-bundle
key: 7442     value: www.mobygames.com/game/xbox-one/world-of-tanks-monster-mega-bundle
key: 7443     value: www.mobygames.com/game/xbox-one/world-of-tanks-monster-mega-bundle-ii
key: 7444     value: www.mobygames.com/game/xbox-one/world-of-tanks-monster-revenant-loaded-bundle
key: 7445     value: www.mobygames.com/game/xbox-one/world-of-tanks-mythical-duo
key: 7446     value: www.mobygames.com/game/xbox-one/world-of-tanks-nightmare-mega
key: 7447     value: www.mobygames.com/game/xbox-one/world-of-tanks-paingods-picks-fv201-ultimate
key: 7448     value: www.mobygames.com/game/xbox-one/world-of-tanks-paingods-picks-m4a2e4-sherman-loaded
key: 7449     value: www.mobygames.com/game/xbox-one/world-of-tanks-paingods-picks-m4a2e4-sherman-ultimate
key: 7450     value: www.mobygames.com/game/xbox-one/wor

In [386]:
for key, value in enumerate(master_list[1404:], 1404):
    print("key: " + str(key) + "     " + "value: " + str(value))

key: 1404     value: www.mobygames.com/game/playstation-4/fat-city_
key: 1405     value: www.mobygames.com/game/playstation-4/fated-the-silent-oath
key: 1406     value: www.mobygames.com/game/playstation-4/fateextella-black-bushin-super-skeleton
key: 1407     value: www.mobygames.com/game/playstation-4/fateextella-british-waiter
key: 1408     value: www.mobygames.com/game/playstation-4/fateextella-stay-night-model-artoria
key: 1409     value: www.mobygames.com/game/playstation-4/fateextella-stay-night-model-gilgamesh
key: 1410     value: www.mobygames.com/game/playstation-4/fateextella-stay-night-model-nameless
key: 1411     value: www.mobygames.com/game/playstation-4/fateextella-the-umbral-star
key: 1412     value: www.mobygames.com/game/playstation-4/fateextella-the-umbral-star-moon-crux-edition
key: 1413     value: www.mobygames.com/game/playstation-4/fat-princess-adventures
key: 1414     value: www.mobygames.com/game/playstation-4/feist
key: 1415     value: www.mobygames.com/game/p

key: 2964     value: www.mobygames.com/game/playstation-4/rocksmith-all-new-2014-edition-dethklok-song-pack
key: 2965     value: www.mobygames.com/game/playstation-4/rocksmith-all-new-2014-edition-dethklok-thunderhorse
key: 2966     value: www.mobygames.com/game/playstation-4/rocksmith-all-new-2014-edition-dick-dale-and-his-del-tones-misir
key: 2967     value: www.mobygames.com/game/playstation-4/rocksmith-all-new-2014-edition-dinosaur-jr-feel-the-pain
key: 2968     value: www.mobygames.com/game/playstation-4/rocksmith-all-new-2014-edition-dio-holy-diver
key: 2969     value: www.mobygames.com/game/playstation-4/rocksmith-all-new-2014-edition-dio-rainbow-in-the-dark
key: 2970     value: www.mobygames.com/game/playstation-4/rocksmith-all-new-2014-edition-dragonforce-through-the-fire-and-
key: 2971     value: www.mobygames.com/game/playstation-4/rocksmith-all-new-2014-edition-dropkick-murphys-im-shipping-up-t
key: 2972     value: www.mobygames.com/game/playstation-4/rocksmith-all-new-2014

key: 3963     value: www.mobygames.com/game/playstation-4/trulon-the-shadow-engine
key: 3964     value: www.mobygames.com/game/playstation-4/tsuih-senkyo
key: 3965     value: www.mobygames.com/game/playstation-4/tumbleseed
key: 3966     value: www.mobygames.com/game/playstation-4/tumblestone
key: 3967     value: www.mobygames.com/game/playstation-4/tumble-vr
key: 3968     value: www.mobygames.com/game/playstation-4/turing-test
key: 3969     value: www.mobygames.com/game/playstation-4/twinbee
key: 3970     value: www.mobygames.com/game/playstation-4/twin-robots
key: 3971     value: www.mobygames.com/game/playstation-4/twisted-metal-black
key: 3972     value: www.mobygames.com/game/playstation-4/typerider
key: 3973     value: www.mobygames.com/game/playstation-4/typoman-deluxe-edition
key: 3974     value: www.mobygames.com/game/playstation-4/typoman-revised
key: 3975     value: www.mobygames.com/game/playstation-4/uefa-euro-2016-pes-2016-pro-evolution-soccer
key: 3976     value: www.moby

key: 4963     value: www.mobygames.com/game/xbox-one/dishonored-2
key: 4964     value: www.mobygames.com/game/xbox-one/dishonored-2-collectors-edition
key: 4965     value: www.mobygames.com/game/xbox-one/dishonored-death-of-the-outsider
key: 4966     value: www.mobygames.com/game/xbox-one/dishonored-definitive-edition
key: 4967     value: www.mobygames.com/game/xbox-one/disney-afternoon-collection
key: 4968     value: www.mobygames.com/game/xbox-one/disney-fantasia-music-evolved
key: 4969     value: www.mobygames.com/game/xbox-one/disney-infinity-30-edition-starter-pack
key: 4970     value: www.mobygames.com/game/xbox-one/disney-infinity-edition-20-marvel-super-heroes
key: 4971     value: www.mobygames.com/game/xbox-one/disney-infinity-edition-20-toy-box-starter-pack
key: 4972     value: www.mobygames.com/game/xbox-one/disneypixar-cars-3-driven-to-win
key: 4973     value: www.mobygames.com/game/xbox-one/divekick-addition-edition
key: 4974     value: www.mobygames.com/game/xbox-one/divi

key: 5962     value: www.mobygames.com/game/xbox-one/nevermind
key: 5963     value: www.mobygames.com/game/xbox-one/neverwinter
key: 5964     value: www.mobygames.com/game/xbox-one/nhl-15
key: 5965     value: www.mobygames.com/game/xbox-one/nhl-16
key: 5966     value: www.mobygames.com/game/xbox-one/nhl-17
key: 5967     value: www.mobygames.com/game/xbox-one/nhl-18
key: 5968     value: www.mobygames.com/game/xbox-one/nhl-18-young-stars-deluxe-edition
key: 5969     value: www.mobygames.com/game/xbox-one/nhl-18-young-stars-edition
key: 5970     value: www.mobygames.com/game/xbox-one/night-in-the-woods
key: 5971     value: www.mobygames.com/game/xbox-one/nightmare-boy
key: 5972     value: www.mobygames.com/game/xbox-one/nightmares-from-the-deep-the-cursed-heart-collectors-edition
key: 5973     value: www.mobygames.com/game/xbox-one/ninja-pizza-girl
key: 5974     value: www.mobygames.com/game/xbox-one/ninja-shodown
key: 5975     value: www.mobygames.com/game/xbox-one/not-a-hero
key: 5976  

key: 6962     value: www.mobygames.com/game/xbox-one/sniper-ghost-warrior-3
key: 6963     value: www.mobygames.com/game/xbox-one/soccer-brawl
key: 6964     value: www.mobygames.com/game/xbox-one/soda-drinker-pro
key: 6965     value: www.mobygames.com/game/xbox-one/solar-shifter-ex
key: 6966     value: www.mobygames.com/game/xbox-one/solitaire_____
key: 6967     value: www.mobygames.com/game/xbox-one/solus-project
key: 6968     value: www.mobygames.com/game/xbox-one/soma
key: 6969     value: www.mobygames.com/game/xbox-one/so-many-me
key: 6970     value: www.mobygames.com/game/xbox-one/songbringer
key: 6971     value: www.mobygames.com/game/xbox-one/song-of-the-deep
key: 6972     value: www.mobygames.com/game/xbox-one/song-of-the-deep-collectors-edition
key: 6973     value: www.mobygames.com/game/xbox-one/sonic-forces
key: 6974     value: www.mobygames.com/game/xbox-one/sonic-forces-digital-bonus-edition
key: 6975     value: www.mobygames.com/game/xbox-one/sonic-mania
key: 6976     valu